In [52]:
import hvplot.pandas
import pandas as pd
import requests

geoapify_key = "9c9ea6c4396f4a19a5e3308aa6b09c9e"
#from api_keys import geoapify_key

In [53]:
city_data_df = pd.read_csv("Starter_Code/output_data/cities.csv")
city_data_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,vodnyy,63.5046,53.4095,37.63,85,90,6.31,RU,1729130090
1,1,margaret river,-33.9500,115.0667,60.78,57,8,9.95,AU,1729130091
2,2,iqaluit,63.7506,-68.5145,33.53,86,100,16.11,CA,1729130092
3,3,walvis bay,-22.9575,14.5053,50.95,93,0,6.91,NaN,1729130093
4,4,tuy hoa,13.0833,109.3000,83.50,74,96,3.00,VN,1729130094
5,5,udachny,66.4167,112.4000,9.05,92,97,11.10,RU,1729130096
6,6,bredasdorp,-34.5322,20.0403,56.25,85,10,7.72,ZA,1729130098
7,7,qaqortoq,60.7167,-46.0333,31.28,54,100,8.68,GL,1729130099
8,8,saldanha,-33.0117,17.9442,57.25,96,6,7.99,ZA,1729130100
9,9,el bauga,18.2620,33.9081,85.01,23,0,5.82,SD,1729130101


In [54]:
%%capture --no-display

humidity = city_data_df["Humidity"].astype(int)
city_map = city_data_df.hvplot.points("Lng",
                                     "Lat",
                                     geo = True,
                                     tiles = "OSM",
                                     size = "Humidity",
                                     color = "City",
                                     alpha = 0.6
)
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [55]:
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 45) & (city_data_df["Max Temp"] > 30)]
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 6.5)]
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]

city_data_df = city_data_df.dropna()
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
122,122,roma,43.2128,-75.4557,38.66,94,0,0.00,US,1729130230
135,135,qusmuryn,52.4503,64.6311,32.81,59,0,5.53,KZ,1729130245
490,490,kirksville,40.1948,-92.5833,41.59,68,0,3.44,US,1729130661
507,507,harrisonville,38.6533,-94.3488,42.58,61,0,0.00,US,1729130576
541,541,arkhara,49.4212,130.0842,41.36,45,0,6.35,RU,1729130719


In [56]:
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":"9c9ea6c4396f4a19a5e3308aa6b09c9e"
}

print("Starting hotel search")
for index, row in city_data_df.iterrows():
    lat = row["Lat"]
    lon = row["Lng"]
    
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    base_url = "https://api.geoapify.com/v2/places"
    name_address = requests.get(base_url, params=params)
    name_address = name_address.json()
    
    try:
        city_data_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        city_data_df.loc[index, "Hotel Name"] = "No hotel found"
        
    print(f"{city_data_df.loc[index, 'City']} - nearest hotel: {city_data_df.loc[index, 'Hotel Name']}")
city_data_df

Starting hotel search
roma - nearest hotel: Hampton Inn
qusmuryn - nearest hotel: No hotel found
kirksville - nearest hotel: Quality Inn
harrisonville - nearest hotel: Knights Inn Harrisonville
arkhara - nearest hotel: No hotel found


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
122,122,roma,43.2128,-75.4557,38.66,94,0,0.00,US,1729130230,Hampton Inn
135,135,qusmuryn,52.4503,64.6311,32.81,59,0,5.53,KZ,1729130245,No hotel found
490,490,kirksville,40.1948,-92.5833,41.59,68,0,3.44,US,1729130661,Quality Inn
507,507,harrisonville,38.6533,-94.3488,42.58,61,0,0.00,US,1729130576,Knights Inn Harrisonville
541,541,arkhara,49.4212,130.0842,41.36,45,0,6.35,RU,1729130719,No hotel found


In [57]:
%%capture --no-display

map_plot_2 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75,
    hover_cols=["Hotel Name", "Country"]
)
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)